In [1]:
import pandas as pd
import psycopg2
import json

In [2]:
# Load the configuration file

file_path = 'config.txt'

# Open and load the JSON data
with open(file_path, 'r') as file:
    config = json.load(file)

# Now `data` is a Python dictionary (or list, depending on the file's structure)

In [3]:
# Pull the required configuration fields from the file

host = config['host']
dbname = config['dbName']
user = config['user']
password = config['password']
port = config['port']

In [5]:
# Define connection parameters
conn = psycopg2.connect( 
    host = host, # pgAdmin 4 > server properties > connection
    dbname = dbname, # pgAdmin 4 > database properties
    user = user, 
    password = password,
    port = port # pgAdmin 4 > server properties > connection
)

# Fetch all data from market_summary table in the database
query = '''
    SELECT
        ticker,
        open,
        high,
        low,
        close,
        volume,
        vwap,
        timestamp,
        transactions,
        otc,
        dw_last_updated
    FROM market_summary
    ORDER BY timestamp
'''

# Save the query to dataframe
market_summary_df = pd.read_sql_query(query, conn)

# Cleaning up
conn.close()

/var/folders/9v/68m5q1k140v1210s62chp_zr0000gn/T/ipykernel_7038/1161103452.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  market_summary_df = pd.read_sql_query(query, conn)


In [6]:
market_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132814 entries, 0 to 132813
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ticker           132814 non-null  object        
 1   open             132814 non-null  float64       
 2   high             132814 non-null  float64       
 3   low              132814 non-null  float64       
 4   close            132814 non-null  float64       
 5   volume           132814 non-null  float64       
 6   vwap             132814 non-null  object        
 7   timestamp        132814 non-null  datetime64[ns]
 8   transactions     131638 non-null  float64       
 9   otc              0 non-null       object        
 10  dw_last_updated  132814 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(6), object(3)
memory usage: 11.1+ MB


In [7]:
# Drop the otc column as its missing throughout the dataset

market_summary_df = market_summary_df.drop(columns = ['otc'])

# Fix vwap column datatype

market_summary_df.vwap = market_summary_df['vwap'].astype('float')

market_summary_df.head()

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,dw_last_updated
0,TINT,29.1303,29.1303,29.1303,29.1303,4.0,29.1200,2025-05-12 20:00:00,1.0,2025-05-28 14:14:26
1,USNZ,36.7094,36.7094,36.7094,36.7094,24.0,36.5353,2025-05-12 20:00:00,10.0,2025-05-28 14:14:26
2,SAWS,19.5412,19.5412,19.5412,19.5412,8.0,19.6050,2025-05-12 20:00:00,4.0,2025-05-28 14:14:26
3,ROPE,24.2197,24.2197,24.2197,24.2197,0.0,NaN,2025-05-12 20:00:00,NaN,2025-05-28 14:14:26
4,HKND,31.6278,31.6278,31.6278,31.6278,150.0,31.5016,2025-05-12 20:00:00,13.0,2025-05-28 14:14:26


In [8]:
market_summary_df.describe(include='all')

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,dw_last_updated
count,132814,132814.000000,132814.000000,132814.000000,132814.000000,1.328140e+05,131638.000000,132814,1.316380e+05,132814
unique,11379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,TINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,117.532892,118.314890,116.602017,117.385040,1.627386e+06,117.900175,2025-05-19 23:58:06.373424640,9.266558e+03,2025-05-28 23:21:25.340551168
min,NaN,0.000600,0.000700,0.000100,0.000200,0.000000e+00,0.000548,2025-05-12 20:00:00,1.000000e+00,2025-05-27 19:06:24
25%,NaN,8.560000,8.770000,8.380000,8.560000,9.672250e+03,8.412275,2025-05-14 20:00:00,8.400000e+01,2025-05-28 11:09:13
50%,NaN,24.030000,24.199850,23.900000,24.052200,8.100850e+04,23.912300,2025-05-19 20:00:00,6.040000e+02,2025-05-29 09:54:59
75%,NaN,42.680000,43.010225,42.390000,42.720000,5.858755e+05,42.595875,2025-05-23 20:00:00,6.579000e+03,2025-05-29 09:59:51
max,NaN,779848.570000,779853.260000,767000.000000,772100.000000,2.454362e+09,771402.220700,2025-05-28 20:00:00,2.570463e+06,2025-05-29 10:00:18


### 1. Which stocks had the largest positive/negative daily returns?

### 2. Stocks with highest/lowest volume?

### 3. Which stocks were most volatile depending on the difference between high and low price?